# Data Cleaning

Summary of changes made in this section:
- Training Dataset:
    - Set index to datetime
    - Combine number of mosquitos caught by a trap on the same day which are presented in different rows (due to number of mosquitos > 50). Drop original duplicated columns.
    - Create a feature to account for 'week_of_year'
    - Drop 'Address', 'Block', 'Street', 'AddressNumberAndStreet', 'AddressAccuracy' as those features are already represented by the Longitude and Latitude


- Testing Dataset:
    - Set index to datetime
    - Create a feature to account for 'week_of_year'
    - Drop 'Address', 'Block', 'Street', 'AddressNumberAndStreet', 'AddressAccuracy' as those features are already represented by the Longitude and Latitude
    

- Weather Dataset:
    - Set index to datetime
    - Drop 'Depth', 'Water1', 'SnowFall', 'StnPressure', 'SeaLevel', 'ResultDir', 'ResultSpeed', 'CodeSum' as those features are not related to the West Nile Virus presence
    - Obtain 'Tavg' data based on 'Tmin' and 'Tmax' for Station 2
    - Convert 'Depart' column into numeric
    - Impute missing values on 'WetBulb' column and convert it to numeric
    - Impute missing values on 'Precipitation' column and convert it to numeric
    - Impute missing values on 'Heat' column from Station 2 with values from Station 1
    - Impute missing values on 'Cool' and 'Heat' columns from Station 2 with values from Station 1
    - Convert 'Sunset' and 'Sunrise' columns from Station 1 and use it as the representative for the entire city
    - Impute missing values on 'AvgSpeed' column from Station 2 with values from Station 1
    - Combine values from both stations
    
    
- Spray Dataset:
    - Set index to datetime

## Packages Import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from datetime import datetime, timedelta

pd.set_option('display.max_columns', None)

In [2]:
# Change this variable to 0 if you use jupyter notebook, or 1 if you use google colab
use_colab = 0

In [3]:
import os
if use_colab:
    from google.colab import files
    import io

if os.path.exists('dataset'):
    print('Dataset folder exists')
else:
    print('Dataset folder not exists. A dataset folder will be created.')
    os.mkdir('dataset')

Dataset folder exists


## Training Dataset

In [4]:
def file_exists(parent_directory, file_name):
    path = os.path.join(parent_directory, file_name)
    if os.path.exists(path):
        return 1
    else:
        return 0

In [5]:
def read_csv_file(use_colab, file_name):
    if file_exists('dataset', file_name):
        return pd.read_csv('dataset/'+file_name)
    else:
        print("Please upload the file or create one by running previous section's csv export.")
        if use_colab:
            print("Please upload file: {}".format(file_name))
            upload_file = files.upload()
            return pd.read_csv(io.BytesIO(upload_file[file_name]))

In [7]:
train = read_csv_file(use_colab, 'train.csv')
train.head(2)

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0


As the data recording is performed such a way that a new row will be generated if there is more than 50 mosquitos on a trap being caught, we will combined these rows into a row for subsequent analysis.

In [8]:
# Count the number of mosquito caught in a trap on the same day for each mosquito species
no_of_mosq = train.groupby(['Date', 'Trap', 'Species'])[['NumMosquitos', 'WnvPresent']].sum()
no_of_mosq.head(5)

NumMosquitos  WnvPresent
Date       Trap Species                                         
2007-05-29 T002 CULEX PIPIENS/RESTUANS             1           0
                CULEX RESTUANS                     1           0
           T007 CULEX RESTUANS                     1           0
           T015 CULEX PIPIENS/RESTUANS             1           0
                CULEX RESTUANS                     4           0

In [9]:
# Impute the value of total mosquito caught
train = train.merge(no_of_mosq, how='left', 
                    left_on=['Date', 'Trap', 'Species'], 
                    right_on=['Date', 'Trap', 'Species'])
train.head(2)

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos_x,WnvPresent_x,NumMosquitos_y,WnvPresent_y
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0,1,0


In [10]:
# If there is a presence of infected mosquito on the trap, it is considered as 1 regardless the number
train['WnvPresent_y']=train.apply(lambda x: 1 if x['WnvPresent_y']>=1 else 0, axis=1)

# Drop the rows containing the same number of mosquito caught based on total mosquito caught
train.drop_duplicates(subset=['Date', 'Species', 'Trap'], keep='first', inplace=True)

# Drop extra columns for original Num of Mosquitos and WnvPresent
train.drop(columns=['NumMosquitos_x', 'WnvPresent_x'], axis=1, inplace=True)

# Rename the column
train.rename(columns={'NumMosquitos_y':'NumMosquitos', 'WnvPresent_y':'WnvPresent'}, inplace=True)

train.head(2)

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0


In [11]:
# Set Date column as index
## Write function to be run on both train and test set later on
def datetime_setter(df):    
    df['Date'] = pd.to_datetime(df['Date'], format=("%Y-%m-%d")).copy()
    df.set_index(keys=['Date'], inplace=True)
    
datetime_setter(train)
train.head(2)

,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
Date,,,,,,,,,,,
2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0
2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0


In [12]:
# Break down the datetime to extract important features
## Write the function to be run on both train and test set later on
def week_of_year_generator(df):
    df['week_of_year'] = df.index.isocalendar().week
    
week_of_year_generator(train)
train.head(2)

,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,week_of_year
Date,,,,,,,,,,,,
2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0,22
2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0,22


Our data contains many columns to represent locations which are redundant. We will remove it and use Latitude and Longitude location to identify precise position of the traps.

In [13]:
# List down the columns to drop
columns_to_drop = ['Address', 'Block', 'Street', 'AddressNumberAndStreet', 'AddressAccuracy']

## Write a function to be run on both train and test set later on
def drop_columns(df, columns_to_drop):
    df.drop(columns=columns_to_drop, inplace=True)
    
drop_columns(train, columns_to_drop)
train.head(2)

,Species,Trap,Latitude,Longitude,NumMosquitos,WnvPresent,week_of_year
Date,,,,,,,
2007-05-29,CULEX PIPIENS/RESTUANS,T002,41.95469,-87.800991,1,0,22
2007-05-29,CULEX RESTUANS,T002,41.95469,-87.800991,1,0,22


## Test Dataset

In [14]:
test = read_csv_file(use_colab, 'test.csv')
test.head(2)

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9


In [15]:
datetime_setter(test)
test.head(2)

,Id,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy
Date,,,,,,,,,,
2008-06-11,1,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
2008-06-11,2,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9


In [16]:
week_of_year_generator(test)
test.head(2)

,Id,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,week_of_year
Date,,,,,,,,,,,
2008-06-11,1,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,24
2008-06-11,2,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,24


In [17]:
drop_columns(test, columns_to_drop)
test.head(2)

,Id,Species,Trap,Latitude,Longitude,week_of_year
Date,,,,,,
2008-06-11,1,CULEX PIPIENS/RESTUANS,T002,41.95469,-87.800991,24
2008-06-11,2,CULEX RESTUANS,T002,41.95469,-87.800991,24


## Weather Dataset

In [18]:
weather = read_csv_file(use_colab, 'weather.csv')
weather.head(2)

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,0448,1849,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,-,-,,M,M,M,0.00,29.18,29.82,2.7,25,9.6


In [19]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Station      2944 non-null   int64  
 1   Date         2944 non-null   object 
 2   Tmax         2944 non-null   int64  
 3   Tmin         2944 non-null   int64  
 4   Tavg         2944 non-null   object 
 5   Depart       2944 non-null   object 
 6   DewPoint     2944 non-null   int64  
 7   WetBulb      2944 non-null   object 
 8   Heat         2944 non-null   object 
 9   Cool         2944 non-null   object 
 10  Sunrise      2944 non-null   object 
 11  Sunset       2944 non-null   object 
 12  CodeSum      2944 non-null   object 
 13  Depth        2944 non-null   object 
 14  Water1       2944 non-null   object 
 15  SnowFall     2944 non-null   object 
 16  PrecipTotal  2944 non-null   object 
 17  StnPressure  2944 non-null   object 
 18  SeaLevel     2944 non-null   object 
 19  Result

After some analysis, we found that some of these columns are not predictive in nature to the presence of West Nile Virus. These columns will be dropped as they will not improve our prediction accuracy:
- Depth
- Water1
- Snowfall
- StnPressure
- SeaLevel
- ResultDir
- ResultSpeed

In [20]:
# Drop a few columns that we will not be using subsequently
weather.drop(columns=['Depth', 'Water1', 'SnowFall', 'StnPressure',
                      'SeaLevel', 'ResultDir', 'ResultSpeed', 'CodeSum'],
            inplace=True)

Also notice that there are some columns which have a non-numeric data type when they are supposed to be numeric according to the data dictionary. These cells are:
- Tavg
- Depart
- WetBulb
- PrecipTotal
- Heat
- Cool
- Sunset
- Sunrise
- AvgSpeed

These columns may contain cells with missing values that is represented by empty strings or some string representation of missing value. As we have information from two weather stations, we can use value obtained from one station to fill up the value on the other station, accordingly. Hence, we will break the dataframe into two by station.

In [21]:
# Split the weather data into two dataframe representing each weather station input
weather_1 = weather[weather['Station']==1].copy()
weather_2 = weather[weather['Station']==2].copy()

In [22]:
# Location of each station
stat_1_loc = [41.995, -87.933]
stat_2_loc = [41.786, -87.752]

In [23]:
# Convert index to time index
datetime_setter(weather_1)
datetime_setter(weather_2)

### Individual Feature Cleaning

#### Average Temperature

In [24]:
# Create a function to detect non-numeric value in the dataframe
def find_non_numeric(df, column):
    return len(df) - df[column].str.isnumeric().sum()

In [25]:
print("Number of non-numeric on Station 1 dataset: {}".format(find_non_numeric(weather_1, 'Tavg')))
print("Number of non-numeric on Station 2 dataset: {}".format(find_non_numeric(weather_2, 'Tavg')))

Number of non-numeric on Station 1 dataset: 0
Number of non-numeric on Station 2 dataset: 11


Only station 2 has empty data. Fortunately, we have Tmin and Tmax data to deduce the Tavg data.

In [26]:
# If Tavg cell is not a number, change it to NaN
weather_2['Tavg'] = pd.to_numeric(weather_2['Tavg'], errors = 'coerce')

# Now that they are NaN, bill with average of Tmax and Tmin
weather_2['Tavg'] = weather_2.apply(lambda x: int((x['Tmax'] + x['Tmin'])/2) 
                                    if np.isnan(x['Tavg']) else int(x['Tavg']), axis=1)

In [27]:
# Cast value in 'PrecipTotal' column on both Stations data to numeric
weather_1['Tavg'] = pd.to_numeric(weather_1['Tavg'], errors = 'coerce')
weather_2['Tavg'] = pd.to_numeric(weather_2['Tavg'], errors = 'coerce')

#### Depart

In [28]:
print("Number of non-numeric elements on Station 1 dataset: {}".format(find_non_numeric(weather_1, 'Depart')))
print("Number of non-numeric elements on Station 2 dataset: {}".format(find_non_numeric(weather_2, 'Depart')))

Number of non-numeric elements on Station 1 dataset: 1271
Number of non-numeric elements on Station 2 dataset: 1472


In [29]:
weather_1['Depart'].unique()

array(['14', '-3', ' 2', ' 4', ' 5', '10', '12', '13', '-2', '15', '11',
       '-4', '-6', ' 8', '-5', ' 1', ' 9', ' 6', '-9', '-8', ' 3', ' 0',
       '-1', '-7', ' 7', '-14', '18', '16', '22', '21', '20', '-10',
       '-16', '17', '-12', '-11', '-15', '-17', '19', '-13', '23'],
      dtype=object)

In [30]:
weather_2['Depart'].unique()

array(['M'], dtype=object)

There are multiple empty values on both stations. Looking closer on the unique values of the data, we can see that all values on Station 1 is numerical while values on station 2 is missing. We will subsequently take Station 1 to be the measure for whole city.

In [31]:
# Remove space in 'Depart' column in station 1 data and cast it to integer
weather_1['Depart'] = pd.to_numeric(weather_1['Depart'], errors = 'coerce')

#### Wet Bulb

In [32]:
print("Number of non-numeric elements on station 1 dataset: {}".format(find_non_numeric(weather_1, 'WetBulb')))
print("Number of non-numeric elements on station 2 dataset: {}".format(find_non_numeric(weather_2, 'WetBulb')))

Number of non-numeric elements on station 1 dataset: 3
Number of non-numeric elements on station 2 dataset: 1


In [33]:
pd.DataFrame(data = [weather_1['WetBulb'], weather_2['WetBulb']], index=['Station 1', 'Station 2'])

Date,2007-05-01,2007-05-02,2007-05-03,2007-05-04,2007-05-05,2007-05-06,2007-05-07,2007-05-08,2007-05-09,2007-05-10,2007-05-11,2007-05-12,2007-05-13,2007-05-14,2007-05-15,2007-05-16,2007-05-17,2007-05-18,2007-05-19,2007-05-20,2007-05-21,2007-05-22,2007-05-23,2007-05-24,2007-05-25,2007-05-26,2007-05-27,2007-05-28,2007-05-29,2007-05-30,2007-05-31,2007-06-01,2007-06-02,2007-06-03,2007-06-04,2007-06-05,2007-06-06,2007-06-07,2007-06-08,2007-06-09,2007-06-10,2007-06-11,2007-06-12,2007-06-13,2007-06-14,2007-06-15,2007-06-16,2007-06-17,2007-06-18,2007-06-19,2007-06-20,2007-06-21,2007-06-22,2007-06-23,2007-06-24,2007-06-25,2007-06-26,2007-06-27,2007-06-28,2007-06-29,2007-06-30,2007-07-01,2007-07-02,2007-07-03,2007-07-04,2007-07-05,2007-07-06,2007-07-07,2007-07-08,2007-07-09,2007-07-10,2007-07-11,2007-07-12,2007-07-13,2007-07-14,2007-07-15,2007-07-16,2007-07-17,2007-07-18,2007-07-19,2007-07-20,2007-07-21,2007-07-22,2007-07-23,2007-07-24,2007-07-25,2007-07-26,2007-07-27,2007-07-28,2007-07-29,2007-07-30,2007-07-31,2007-08-01,2007-08-02,2007-08-03,2007-08-04,2007-08-05,2007-08-06,2007-08-07,2007-08-08,2007-08-09,2007-08-10,2007-08-11,2007-08-12,2007-08-13,2007-08-14,2007-08-15,2007-08-16,2007-08-17,2007-08-18,2007-08-19,2007-08-20,2007-08-21,2007-08-22,2007-08-23,2007-08-24,2007-08-25,2007-08-26,2007-08-27,2007-08-28,2007-08-29,2007-08-30,2007-08-31,2007-09-01,2007-09-02,2007-09-03,2007-09-04,2007-09-05,2007-09-06,2007-09-07,2007-09-08,2007-09-09,2007-09-10,2007-09-11,2007-09-12,2007-09-13,2007-09-14,2007-09-15,2007-09-16,2007-09-17,2007-09-18,2007-09-19,2007-09-20,2007-09-21,2007-09-22,2007-09-23,2007-09-24,2007-09-25,2007-09-26,2007-09-27,2007-09-28,2007-09-29,2007-09-30,2007-10-01,2007-10-02,2007-10-03,2007-10-04,2007-10-05,2007-10-06,2007-10-07,2007-10-08,2007-10-09,2007-10-10,2007-10-11,2007-10-12,2007-10-13,2007-10-14,2007-10-15,2007-10-16,2007-10-17,2007-10-18,2007-10-19,2007-10-20,2007-10-21,2007-10-22,2007-10-23,2007-10-24,2007-10-25,2007-10-26,2007-10-27,2007-10-28,2007-10-29,2007-10-30,2007-10-31,2008-05-01,2008-05-02,2008-05-03,2008-05-04,2008-05-05,2008-05-06,2008-05-07,2008-05-08,2008-05-09,2008-05-10,2008-05-11,2008-05-12,2008-05-13,2008-05-14,2008-05-15,2008-05-16,2008-05-17,2008-05-18,2008-05-19,2008-05-20,2008-05-21,2008-05-22,2008-05-23,2008-05-24,2008-05-25,2008-05-26,2008-05-27,2008-05-28,2008-05-29,2008-05-30,2008-05-31,2008-06-01,2008-06-02,2008-06-03,2008-06-04,2008-06-05,2008-06-06,2008-06-07,2008-06-08,2008-06-09,2008-06-10,2008-06-11,2008-06-12,2008-06-13,2008-06-14,2008-06-15,2008-06-16,2008-06-17,2008-06-18,2008-06-19,2008-06-20,2008-06-21,2008-06-22,2008-06-23,2008-06-24,2008-06-25,2008-06-26,2008-06-27,2008-06-28,2008-06-29,2008-06-30,2008-07-01,2008-07-02,2008-07-03,2008-07-04,2008-07-05,2008-07-06,2008-07-07,2008-07-08,2008-07-09,2008-07-10,2008-07-11,2008-07-12,2008-07-13,2008-07-14,2008-07-15,2008-07-16,2008-07-17,2008-07-18,2008-07-19,2008-07-20,2008-07-21,2008-07-22,2008-07-23,2008-07-24,2008-07-25,2008-07-26,2008-07-27,2008-07-28,2008-07-29,2008-07-30,2008-07-31,2008-08-01,2008-08-02,2008-08-03,2008-08-04,2008-08-05,2008-08-06,2008-08-07,2008-08-08,2008-08-09,2008-08-10,2008-08-11,2008-08-12,2008-08-13,2008-08-14,2008-08-15,2008-08-16,2008-08-17,2008-08-18,2008-08-19,2008-08-20,2008-08-21,2008-08-22,2008-08-23,2008-08-24,2008-08-25,2008-08-26,2008-08-27,2008-08-28,2008-08-29,2008-08-30,2008-08-31,2008-09-01,2008-09-02,2008-09-03,2008-09-04,2008-09-05,2008-09-06,2008-09-07,2008-09-08,2008-09-09,2008-09-10,2008-09-11,2008-09-12,2008-09-13,2008-09-14,2008-09-15,2008-09-16,2008-09-17,2008-09-18,2008-09-19,2008-09-20,2008-09-21,2008-09-22,2008-09-23,2008-09-24,2008-09-25,2008-09-26,2008-09-27,2008-09-28,2008-09-29,2008-09-30,2008-10-01,2008-10-02,2008-10-03,2008-10-04,2008-10-05,2008-10-06,2008-10-07,2008-10-08,2008-10-09,2008-10-10,2008-10-11,2008-10-12,2008-10-13,2008-10-14,2008-10-15,2008-10-16,2008-10-17,2008-10-18,2008-10-19,2008-10-20,2008-10-21,2008-10-22,2008-10-23,2008-10-24,2008-10-25,2008-10-26,20

As there are only 4 empty data (3 from Station 1 and 1 from Station 2) and generally the value of Wet Bulb temperature does not vary between the stations, we will impute the value from the station to one another.

In [34]:
# Impute M value in Station 1 with Station 2 value
for index in weather_1[weather_1['WetBulb'] == 'M'].index:
    weather_1.at[index, 'WetBulb'] = weather_2.loc[index]['WetBulb']
# Impute M value in Station 1 with Station 2 value
for index in weather_2[weather_2['WetBulb'] == 'M'].index:
    weather_2.at[index, 'WetBulb'] = weather_1.loc[index]['WetBulb']

In [35]:
# Cast value in 'PrecipTotal' column on both Stations data to numeric
weather_1['WetBulb'] = pd.to_numeric(weather_1['WetBulb'], errors = 'coerce')
weather_2['WetBulb'] = pd.to_numeric(weather_2['WetBulb'], errors = 'coerce')

#### Precipitation Total

In [36]:
print("Number of non-numeric elements on Station 1 dataset: {}".format(find_non_numeric(weather_1, 'PrecipTotal')))
print("Number of non-numeric elements on Station 2 dataset: {}".format(find_non_numeric(weather_2, 'PrecipTotal')))

Number of non-numeric elements on Station 1 dataset: 1472
Number of non-numeric elements on Station 2 dataset: 1472


In [37]:
weather_1['PrecipTotal'].unique()

array(['0.00', '  T', '0.13', '0.38', '0.14', '0.07', '0.02', '1.01',
       '0.04', '0.01', '0.19', '0.39', '0.42', '0.28', '0.58', '0.05',
       '0.16', '0.15', '0.40', '0.30', '0.43', '0.11', '1.55', '0.17',
       '0.03', '1.43', '0.26', '1.31', '0.06', '0.29', '0.41', '0.23',
       '0.09', '1.33', '0.91', '0.37', '2.35', '0.24', '0.34', '0.18',
       '0.36', '0.27', '0.33', '1.28', '0.76', '0.95', '0.08', '0.12',
       '0.48', '0.52', '0.64', '1.24', '0.20', '0.72', '0.73', '1.61',
       '0.74', '1.05', '2.43', '2.90', '1.23', '6.64', '1.44', '0.82',
       '0.80', '0.32', '0.67', '0.55', '1.73', '0.44', '0.45', '3.97',
       '0.25', '0.68', '0.63', '0.35', '0.93', '0.22', '0.75', '0.89',
       '0.21', '0.46', '1.03', '0.60', '0.81', '0.97', '1.49', '1.02',
       '0.59', '0.87', '0.85', '0.54', '0.79', '2.79', '3.64', '0.10',
       '0.31', '2.24', '1.06', '1.88', '6.86', '0.71', '0.70', '1.11',
       '0.65', '1.21', '0.77', '1.34', '0.47', '1.12', '0.57', '0.51',
       

In [38]:
weather_2['PrecipTotal'].unique()

array(['0.00', '  T', '0.02', '0.60', '0.07', '0.11', '0.09', '0.28',
       '0.08', '0.53', '0.21', '0.32', '0.31', '0.27', '0.16', '0.93',
       '0.34', '0.35', 'M', '0.66', '0.24', '0.92', '0.01', '0.89',
       '0.04', '0.97', '0.05', '0.06', '0.46', '0.23', '0.45', '0.83',
       '0.42', '1.33', '0.48', '0.88', '1.96', '0.20', '0.25', '0.67',
       '0.14', '0.17', '0.19', '0.15', '0.03', '0.41', '0.74', '0.71',
       '1.46', '0.26', '0.30', '0.22', '1.31', '0.29', '0.12', '0.43',
       '0.65', '1.22', '0.13', '0.50', '0.59', '2.68', '1.01', '0.62',
       '3.07', '1.75', '0.86', '0.63', '1.03', '0.70', '1.38', '1.14',
       '1.07', '0.87', '0.78', '1.12', '0.10', '0.61', '0.54', '1.19',
       '0.44', '0.33', '0.84', '1.29', '1.05', '1.15', '0.72', '0.40',
       '1.57', '1.30', '0.51', '1.23', '0.77', '3.15', '4.73', '0.52',
       '1.60', '0.57', '0.37', '1.90', '0.18', '0.69', '2.17', '0.36',
       '0.49', '0.39', '0.90', '0.38', '0.56', '0.55', '1.02', '3.66',
       '1.

In [39]:
pd.DataFrame(data = [weather_1['PrecipTotal'], weather_2['PrecipTotal']], index=['Station 1', 'Station 2'])

Date,2007-05-01,2007-05-02,2007-05-03,2007-05-04,2007-05-05,2007-05-06,2007-05-07,2007-05-08,2007-05-09,2007-05-10,2007-05-11,2007-05-12,2007-05-13,2007-05-14,2007-05-15,2007-05-16,2007-05-17,2007-05-18,2007-05-19,2007-05-20,2007-05-21,2007-05-22,2007-05-23,2007-05-24,2007-05-25,2007-05-26,2007-05-27,2007-05-28,2007-05-29,2007-05-30,2007-05-31,2007-06-01,2007-06-02,2007-06-03,2007-06-04,2007-06-05,2007-06-06,2007-06-07,2007-06-08,2007-06-09,2007-06-10,2007-06-11,2007-06-12,2007-06-13,2007-06-14,2007-06-15,2007-06-16,2007-06-17,2007-06-18,2007-06-19,2007-06-20,2007-06-21,2007-06-22,2007-06-23,2007-06-24,2007-06-25,2007-06-26,2007-06-27,2007-06-28,2007-06-29,2007-06-30,2007-07-01,2007-07-02,2007-07-03,2007-07-04,2007-07-05,2007-07-06,2007-07-07,2007-07-08,2007-07-09,2007-07-10,2007-07-11,2007-07-12,2007-07-13,2007-07-14,2007-07-15,2007-07-16,2007-07-17,2007-07-18,2007-07-19,2007-07-20,2007-07-21,2007-07-22,2007-07-23,2007-07-24,2007-07-25,2007-07-26,2007-07-27,2007-07-28,2007-07-29,2007-07-30,2007-07-31,2007-08-01,2007-08-02,2007-08-03,2007-08-04,2007-08-05,2007-08-06,2007-08-07,2007-08-08,2007-08-09,2007-08-10,2007-08-11,2007-08-12,2007-08-13,2007-08-14,2007-08-15,2007-08-16,2007-08-17,2007-08-18,2007-08-19,2007-08-20,2007-08-21,2007-08-22,2007-08-23,2007-08-24,2007-08-25,2007-08-26,2007-08-27,2007-08-28,2007-08-29,2007-08-30,2007-08-31,2007-09-01,2007-09-02,2007-09-03,2007-09-04,2007-09-05,2007-09-06,2007-09-07,2007-09-08,2007-09-09,2007-09-10,2007-09-11,2007-09-12,2007-09-13,2007-09-14,2007-09-15,2007-09-16,2007-09-17,2007-09-18,2007-09-19,2007-09-20,2007-09-21,2007-09-22,2007-09-23,2007-09-24,2007-09-25,2007-09-26,2007-09-27,2007-09-28,2007-09-29,2007-09-30,2007-10-01,2007-10-02,2007-10-03,2007-10-04,2007-10-05,2007-10-06,2007-10-07,2007-10-08,2007-10-09,2007-10-10,2007-10-11,2007-10-12,2007-10-13,2007-10-14,2007-10-15,2007-10-16,2007-10-17,2007-10-18,2007-10-19,2007-10-20,2007-10-21,2007-10-22,2007-10-23,2007-10-24,2007-10-25,2007-10-26,2007-10-27,2007-10-28,2007-10-29,2007-10-30,2007-10-31,2008-05-01,2008-05-02,2008-05-03,2008-05-04,2008-05-05,2008-05-06,2008-05-07,2008-05-08,2008-05-09,2008-05-10,2008-05-11,2008-05-12,2008-05-13,2008-05-14,2008-05-15,2008-05-16,2008-05-17,2008-05-18,2008-05-19,2008-05-20,2008-05-21,2008-05-22,2008-05-23,2008-05-24,2008-05-25,2008-05-26,2008-05-27,2008-05-28,2008-05-29,2008-05-30,2008-05-31,2008-06-01,2008-06-02,2008-06-03,2008-06-04,2008-06-05,2008-06-06,2008-06-07,2008-06-08,2008-06-09,2008-06-10,2008-06-11,2008-06-12,2008-06-13,2008-06-14,2008-06-15,2008-06-16,2008-06-17,2008-06-18,2008-06-19,2008-06-20,2008-06-21,2008-06-22,2008-06-23,2008-06-24,2008-06-25,2008-06-26,2008-06-27,2008-06-28,2008-06-29,2008-06-30,2008-07-01,2008-07-02,2008-07-03,2008-07-04,2008-07-05,2008-07-06,2008-07-07,2008-07-08,2008-07-09,2008-07-10,2008-07-11,2008-07-12,2008-07-13,2008-07-14,2008-07-15,2008-07-16,2008-07-17,2008-07-18,2008-07-19,2008-07-20,2008-07-21,2008-07-22,2008-07-23,2008-07-24,2008-07-25,2008-07-26,2008-07-27,2008-07-28,2008-07-29,2008-07-30,2008-07-31,2008-08-01,2008-08-02,2008-08-03,2008-08-04,2008-08-05,2008-08-06,2008-08-07,2008-08-08,2008-08-09,2008-08-10,2008-08-11,2008-08-12,2008-08-13,2008-08-14,2008-08-15,2008-08-16,2008-08-17,2008-08-18,2008-08-19,2008-08-20,2008-08-21,2008-08-22,2008-08-23,2008-08-24,2008-08-25,2008-08-26,2008-08-27,2008-08-28,2008-08-29,2008-08-30,2008-08-31,2008-09-01,2008-09-02,2008-09-03,2008-09-04,2008-09-05,2008-09-06,2008-09-07,2008-09-08,2008-09-09,2008-09-10,2008-09-11,2008-09-12,2008-09-13,2008-09-14,2008-09-15,2008-09-16,2008-09-17,2008-09-18,2008-09-19,2008-09-20,2008-09-21,2008-09-22,2008-09-23,2008-09-24,2008-09-25,2008-09-26,2008-09-27,2008-09-28,2008-09-29,2008-09-30,2008-10-01,2008-10-02,2008-10-03,2008-10-04,2008-10-05,2008-10-06,2008-10-07,2008-10-08,2008-10-09,2008-10-10,2008-10-11,2008-10-12,2008-10-13,2008-10-14,2008-10-15,2008-10-16,2008-10-17,2008-10-18,2008-10-19,2008-10-20,2008-10-21,2008-10-22,2008-10-23,2008-10-24,2008-10-25,2008-10-26,20

First we see that all the Precipitation Total values are not considered as numeric by isnumeric() method. However, we can see that most of them are actually of numeric values. We will attempt to convert them to be numeric value later on using pd.to_numeric() method. However, the more pressing issue is to deal with 'T' (signifying trace value: value very small but not zero ([source](https://en.wikipedia.org/wiki/Trace_(precipitation)))) and 'M' (missing value). We will deal with them separately. 

In [40]:
# For value denoted as 'T' we will convert them to 0.0001
weather_1['PrecipTotal'] = weather_1['PrecipTotal'].apply(lambda x: 0.0001 if x == '  T' else x)
weather_2['PrecipTotal'] = weather_2['PrecipTotal'].apply(lambda x: 0.0001 if x == '  T' else x)

In [41]:
print("Number of elements with missing value on Station 1: {}".format((weather_1['PrecipTotal']=='M').sum()))
print("Number of elements with missing value on Station 2: {}".format((weather_2['PrecipTotal']=='M').sum()))

Number of elements with missing value on Station 1: 0
Number of elements with missing value on Station 2: 2


As there are no empty elements on Station 1, we will impute Station 2 missing value with Station 1 value. 

In [42]:
# Impute M value in Station 1 with Station 2 value
for index in weather_2[weather_2['PrecipTotal'] == 'M'].index:
    weather_2.at[index, 'PrecipTotal'] = weather_1.loc[index]['PrecipTotal']

In [43]:
# Cast value in 'PrecipTotal' column on both Stations data to numeric
weather_1['PrecipTotal'] = pd.to_numeric(weather_1['PrecipTotal'], errors = 'coerce')
weather_2['PrecipTotal'] = pd.to_numeric(weather_2['PrecipTotal'], errors = 'coerce')

#### Heat

In [44]:
print("Number of non-numeric elements on Station 1 dataset: {}".format(find_non_numeric(weather_1, 'Heat')))
print("Number of non-numeric elements on Station 2 dataset: {}".format(find_non_numeric(weather_2, 'Heat')))

Number of non-numeric elements on Station 1 dataset: 0
Number of non-numeric elements on Station 2 dataset: 11


As there are no empty elements on Station 1, we will impute Station 2 missing value with Station 1 value. 

In [45]:
weather_2['Heat'].unique()

array(['0', '13', '7', 'M', '5', '4', '9', '8', '10', '12', '2', '3', '1',
       '15', '14', '6', '16', '17', '20', '11', '18', '19', '24', '26',
       '23', '28', '22', '21', '29', '25'], dtype=object)

In [46]:
# Impute M value in Station 2 with Station 1 value
for index in weather_2[weather_2['Heat'] == 'M'].index:
    weather_2.at[index, 'Heat'] = weather_1.loc[index]['Heat']

In [47]:
# Cast value in 'PrecipTotal' column on both Stations data to numeric
weather_1['Heat'] = pd.to_numeric(weather_1['Heat'], errors = 'coerce')
weather_2['Heat'] = pd.to_numeric(weather_2['Heat'], errors = 'coerce')

#### Cool

In [48]:
print("Number of non-numeric elements on Station 1 dataset: {}".format(find_non_numeric(weather_1, 'Cool')))
print("Number of non-numeric elements on Station 2 dataset: {}".format(find_non_numeric(weather_2, 'Cool')))

Number of non-numeric elements on Station 1 dataset: 1096
Number of non-numeric elements on Station 2 dataset: 1021


In [49]:
weather_1['Cool'].unique()

array([' 2', ' 0', ' 3', ' 4', ' 5', ' 8', ' 6', '10', '11', ' 1', ' 9',
       '12', ' 7', '16', '15', '14', '20', '17', '13', '18', '19', '21',
       '22', '23', '26', '28', '24'], dtype=object)

In [50]:
weather_2['Cool'].unique()

array([' 3', ' 0', 'M', ' 2', ' 5', ' 6', ' 7', ' 4', ' 9', '13', '12',
       '15', '10', '17', '14', '16', ' 8', '11', '19', '18', ' 1', '20',
       '22', '21', '24', '27', '26', '29', '23', '25'], dtype=object)

In [51]:
pd.DataFrame(data = [weather_1['Cool'], weather_2['Cool']], index=['Station 1', 'Station 2'])

Date,2007-05-01,2007-05-02,2007-05-03,2007-05-04,2007-05-05,2007-05-06,2007-05-07,2007-05-08,2007-05-09,2007-05-10,2007-05-11,2007-05-12,2007-05-13,2007-05-14,2007-05-15,2007-05-16,2007-05-17,2007-05-18,2007-05-19,2007-05-20,2007-05-21,2007-05-22,2007-05-23,2007-05-24,2007-05-25,2007-05-26,2007-05-27,2007-05-28,2007-05-29,2007-05-30,2007-05-31,2007-06-01,2007-06-02,2007-06-03,2007-06-04,2007-06-05,2007-06-06,2007-06-07,2007-06-08,2007-06-09,2007-06-10,2007-06-11,2007-06-12,2007-06-13,2007-06-14,2007-06-15,2007-06-16,2007-06-17,2007-06-18,2007-06-19,2007-06-20,2007-06-21,2007-06-22,2007-06-23,2007-06-24,2007-06-25,2007-06-26,2007-06-27,2007-06-28,2007-06-29,2007-06-30,2007-07-01,2007-07-02,2007-07-03,2007-07-04,2007-07-05,2007-07-06,2007-07-07,2007-07-08,2007-07-09,2007-07-10,2007-07-11,2007-07-12,2007-07-13,2007-07-14,2007-07-15,2007-07-16,2007-07-17,2007-07-18,2007-07-19,2007-07-20,2007-07-21,2007-07-22,2007-07-23,2007-07-24,2007-07-25,2007-07-26,2007-07-27,2007-07-28,2007-07-29,2007-07-30,2007-07-31,2007-08-01,2007-08-02,2007-08-03,2007-08-04,2007-08-05,2007-08-06,2007-08-07,2007-08-08,2007-08-09,2007-08-10,2007-08-11,2007-08-12,2007-08-13,2007-08-14,2007-08-15,2007-08-16,2007-08-17,2007-08-18,2007-08-19,2007-08-20,2007-08-21,2007-08-22,2007-08-23,2007-08-24,2007-08-25,2007-08-26,2007-08-27,2007-08-28,2007-08-29,2007-08-30,2007-08-31,2007-09-01,2007-09-02,2007-09-03,2007-09-04,2007-09-05,2007-09-06,2007-09-07,2007-09-08,2007-09-09,2007-09-10,2007-09-11,2007-09-12,2007-09-13,2007-09-14,2007-09-15,2007-09-16,2007-09-17,2007-09-18,2007-09-19,2007-09-20,2007-09-21,2007-09-22,2007-09-23,2007-09-24,2007-09-25,2007-09-26,2007-09-27,2007-09-28,2007-09-29,2007-09-30,2007-10-01,2007-10-02,2007-10-03,2007-10-04,2007-10-05,2007-10-06,2007-10-07,2007-10-08,2007-10-09,2007-10-10,2007-10-11,2007-10-12,2007-10-13,2007-10-14,2007-10-15,2007-10-16,2007-10-17,2007-10-18,2007-10-19,2007-10-20,2007-10-21,2007-10-22,2007-10-23,2007-10-24,2007-10-25,2007-10-26,2007-10-27,2007-10-28,2007-10-29,2007-10-30,2007-10-31,2008-05-01,2008-05-02,2008-05-03,2008-05-04,2008-05-05,2008-05-06,2008-05-07,2008-05-08,2008-05-09,2008-05-10,2008-05-11,2008-05-12,2008-05-13,2008-05-14,2008-05-15,2008-05-16,2008-05-17,2008-05-18,2008-05-19,2008-05-20,2008-05-21,2008-05-22,2008-05-23,2008-05-24,2008-05-25,2008-05-26,2008-05-27,2008-05-28,2008-05-29,2008-05-30,2008-05-31,2008-06-01,2008-06-02,2008-06-03,2008-06-04,2008-06-05,2008-06-06,2008-06-07,2008-06-08,2008-06-09,2008-06-10,2008-06-11,2008-06-12,2008-06-13,2008-06-14,2008-06-15,2008-06-16,2008-06-17,2008-06-18,2008-06-19,2008-06-20,2008-06-21,2008-06-22,2008-06-23,2008-06-24,2008-06-25,2008-06-26,2008-06-27,2008-06-28,2008-06-29,2008-06-30,2008-07-01,2008-07-02,2008-07-03,2008-07-04,2008-07-05,2008-07-06,2008-07-07,2008-07-08,2008-07-09,2008-07-10,2008-07-11,2008-07-12,2008-07-13,2008-07-14,2008-07-15,2008-07-16,2008-07-17,2008-07-18,2008-07-19,2008-07-20,2008-07-21,2008-07-22,2008-07-23,2008-07-24,2008-07-25,2008-07-26,2008-07-27,2008-07-28,2008-07-29,2008-07-30,2008-07-31,2008-08-01,2008-08-02,2008-08-03,2008-08-04,2008-08-05,2008-08-06,2008-08-07,2008-08-08,2008-08-09,2008-08-10,2008-08-11,2008-08-12,2008-08-13,2008-08-14,2008-08-15,2008-08-16,2008-08-17,2008-08-18,2008-08-19,2008-08-20,2008-08-21,2008-08-22,2008-08-23,2008-08-24,2008-08-25,2008-08-26,2008-08-27,2008-08-28,2008-08-29,2008-08-30,2008-08-31,2008-09-01,2008-09-02,2008-09-03,2008-09-04,2008-09-05,2008-09-06,2008-09-07,2008-09-08,2008-09-09,2008-09-10,2008-09-11,2008-09-12,2008-09-13,2008-09-14,2008-09-15,2008-09-16,2008-09-17,2008-09-18,2008-09-19,2008-09-20,2008-09-21,2008-09-22,2008-09-23,2008-09-24,2008-09-25,2008-09-26,2008-09-27,2008-09-28,2008-09-29,2008-09-30,2008-10-01,2008-10-02,2008-10-03,2008-10-04,2008-10-05,2008-10-06,2008-10-07,2008-10-08,2008-10-09,2008-10-10,2008-10-11,2008-10-12,2008-10-13,2008-10-14,2008-10-15,2008-10-16,2008-10-17,2008-10-18,2008-10-19,2008-10-20,2008-10-21,2008-10-22,2008-10-23,2008-10-24,2008-10-25,2008-10-26,20

Values on Station 1 are actually all numerical values with some spaces character on them. We will attempt to cast values on Station 1 first into numerical and observe if there are any errors. 

In [52]:
weather_1['Cool'] = pd.to_numeric(weather_1['Cool'], errors = 'coerce')

In [53]:
print("Number of missing values from Station 2: {}".format((weather_2['Cool'] == 'M').sum()))

Number of missing values from Station 2: 11


Since all values are filled in Station 1 and there are only 11 missing values from Station 2, we will impute the missing values with those from the Station 1. Subsequently, we will convert the values into numerical too. 

In [54]:
# Impute M value in Station 2 with Station 1 value
for index in weather_2[weather_2['Cool'] == 'M'].index:
    weather_2.at[index, 'Cool'] = weather_1.loc[index]['Cool']
    
weather_2['Cool'] = pd.to_numeric(weather_2['Cool'], errors = 'coerce')

#### Sunset

In [55]:
print("Number of non-numeric elements on Station 1 dataset: {}".format(find_non_numeric(weather_1, 'Sunset')))
print("Number of non-numeric elements on Station 2 dataset: {}".format(find_non_numeric(weather_2, 'Sunset')))

Number of non-numeric elements on Station 1 dataset: 0
Number of non-numeric elements on Station 2 dataset: 1472


In [56]:
weather_1['Sunset'].unique()

array(['1849', '1850', '1851', '1852', '1853', '1855', '1856', '1857',
       '1858', '1859', '1860', '1901', '1902', '1903', '1904', '1905',
       '1906', '1907', '1908', '1909', '1910', '1911', '1912', '1913',
       '1914', '1915', '1916', '1917', '1918', '1919', '1920', '1921',
       '1922', '1923', '1924', '1925', '1926', '1927', '1928', '1929',
       '1930', '1931', '1900', '1854', '1848', '1846', '1845', '1843',
       '1842', '1840', '1839', '1837', '1835', '1834', '1832', '1831',
       '1829', '1827', '1826', '1824', '1822', '1821', '1819', '1817',
       '1815', '1814', '1812', '1810', '1809', '1807', '1805', '1803',
       '1760', '1758', '1756', '1755', '1753', '1751', '1749', '1748',
       '1746', '1744', '1742', '1741', '1739', '1737', '1735', '1734',
       '1732', '1730', '1729', '1727', '1725', '1724', '1722', '1720',
       '1719', '1717', '1715', '1714', '1712', '1711', '1709', '1707',
       '1706', '1704', '1703', '1701', '1660', '1658', '1657', '1656',
      

In [57]:
weather_2['Sunset'].unique()

array(['-'], dtype=object)

There are no values provided for Station 2. Therefore, we will use values from Station 1 for the entire city. It was also observed the value of '1860' from Station 1 data which will be converted into '1900'. 

In [58]:
weather_1.loc[(weather_1['Sunset'] == '1860'), 'Sunset'] = '1900'
weather_1['Sunset'] = pd.to_numeric(weather_1['Sunset'], errors = 'coerce')

#### Sunrise

In [59]:
print("Number of non-numeric elements on Station 1 dataset: {}".format(find_non_numeric(weather_1, 'Sunrise')))
print("Number of non-numeric elements on Station 2 dataset: {}".format(find_non_numeric(weather_2, 'Sunrise')))

Number of non-numeric elements on Station 1 dataset: 0
Number of non-numeric elements on Station 2 dataset: 1472


In [60]:
weather_1['Sunrise'].unique()

array(['0448', '0447', '0446', '0444', '0443', '0442', '0441', '0439',
       '0438', '0437', '0436', '0435', '0434', '0433', '0432', '0431',
       '0430', '0429', '0428', '0427', '0426', '0425', '0424', '0423',
       '0422', '0421', '0420', '0419', '0418', '0417', '0416', '0440',
       '0445', '0449', '0450', '0451', '0452', '0453', '0454', '0455',
       '0456', '0457', '0458', '0459', '0500', '0501', '0502', '0503',
       '0504', '0505', '0506', '0508', '0509', '0510', '0511', '0512',
       '0513', '0514', '0515', '0516', '0517', '0518', '0519', '0520',
       '0521', '0522', '0523', '0524', '0525', '0526', '0527', '0528',
       '0529', '0530', '0531', '0532', '0533', '0535', '0536', '0537',
       '0538', '0539', '0540', '0541', '0542', '0543', '0544', '0545',
       '0546', '0547', '0548', '0549', '0550', '0552', '0553', '0554',
       '0555', '0556', '0557', '0558', '0559', '0600', '0602', '0603',
       '0604', '0605', '0606', '0607', '0608', '0610', '0611', '0612',
      

In [61]:
weather_2['Sunrise'].unique()

array(['-'], dtype=object)

Similarly to Sunset column, there are no values provided for Station 2 for Sunrise. Therefore, we will use values from Station 1 for the entire city.

In [62]:
weather_1['Sunrise'] = pd.to_numeric(weather_1['Sunrise'], errors = 'coerce')

#### Average Speed

In [63]:
print("Number of non-numeric elements on Station 1 dataset: {}".format(find_non_numeric(weather_1, 'AvgSpeed')))
print("Number of non-numeric elements on Station 2 dataset: {}".format(find_non_numeric(weather_2, 'AvgSpeed')))

Number of non-numeric elements on Station 1 dataset: 1472
Number of non-numeric elements on Station 2 dataset: 1472


In [64]:
weather_1['AvgSpeed'].unique()

array(['9.2', '13.4', '11.9', '10.8', '12.0', '15.0', '10.5', '5.8',
       '6.2', '4.1', '12.9', '8.1', '17.3', '12.3', '12.2', '11.3', '7.5',
       '11.4', '10.7', '8.2', '10.2', '11.8', '8.8', '7.2', '9.5', '5.3',
       '6.5', '10.6', '9.9', '6.4', '7.4', '6.7', '9.7', '7.6', '23.1',
       '13.8', '5.0', '5.6', '6.9', '8.6', '6.8', '4.9', '6.3', '15.1',
       '8.7', '9.0', '8.0', '7.1', '12.5', '8.3', '7.3', '5.5', '13.1',
       '10.9', '11.1', '13.2', '5.2', '10.1', '9.1', '3.3', '4.0', '6.6',
       '9.4', '7.9', '5.7', '4.3', '5.9', '6.1', '9.6', '6.0', '3.9',
       '5.4', '11.7', '12.1', '13.9', '13.5', '8.5', '12.4', '9.8',
       '14.6', '4.6', '7.0', '12.8', '15.5', '3.5', '18.9', '16.7',
       '18.2', '14.5', '14.1', '4.2', '10.4', '14.3', '11.5', '12.7',
       '12.6', '17.7', '7.8', '15.9', '14.7', '8.4', '19.3', '4.8',
       '14.0', '21.5', '7.7', '10.0', '16.3', '11.2', '5.1', '9.3',
       '10.3', '4.4', '3.8', '2.9', '2.7', '17.8', '14.4', '16.4', '11.0',
     

In [65]:
weather_2['AvgSpeed'].unique()

array(['9.6', '13.4', '13.2', '10.4', '11.5', '14.5', '9.9', '5.4', '5.9',
       '3.9', '12.8', '13.0', '7.6', '14.6', '10.8', '11.8', '12.2',
       '5.5', '11.4', '9.7', '7.8', '10.6', '11.2', '15.2', '8.2', '9.2',
       '5.2', '7.4', '10.1', '5.8', '7.2', '8.1', '20.7', '12.4', '4.7',
       '8.4', '10.0', 'M', '6.4', '4.1', '6.1', '12.6', '11.3', '7.7',
       '6.0', '8.5', '13.3', '6.2', '8.8', '6.9', '12.7', '10.2', '13.9',
       '6.5', '6.6', '14.4', '3.5', '4.9', '7.0', '10.7', '4.6', '5.0',
       '3.7', '6.3', '9.0', '7.1', '5.7', '7.5', '9.1', '7.3', '8.3',
       '10.9', '4.4', '6.7', '9.4', '13.7', '11.0', '10.5', '8.9', '10.3',
       '15.5', '17.2', '18.0', '11.7', '16.8', '7.9', '17.3', '14.7',
       '9.3', '15.0', '5.6', '11.1', '17.8', '14.1', '12.0', '17.4',
       '3.8', '12.3', '4.3', '13.8', '19.5', '15.1', '13.5', '8.6', '8.0',
       '6.8', '3.6', '4.0', '13.1', '9.5', '3.2', '4.5', '12.1', '3.1',
       '4.8', '8.7', '5.3', '18.4', '14.8', '2.6', '4.2', '2.

As there are no missing values on Station 1, we will convert Station 1 value first into numerical. 

In [66]:
weather_1['AvgSpeed'] = pd.to_numeric(weather_1['AvgSpeed'], errors = 'coerce')
print("Number of missing values from Station 2: {}".format((weather_2['AvgSpeed'] == 'M').sum()))

Number of missing values from Station 2: 3


Since all values are filled in Station 1 and there are only 3 missing values from Station 2, we will impute the missing values with those from the Station 1. Subsequently, we will convert the values into numerical too. 

In [67]:
# Impute M value in Station 2 with Station 1 value
for index in weather_2[weather_2['AvgSpeed'] == 'M'].index:
    weather_2.at[index, 'AvgSpeed'] = weather_1.loc[index]['AvgSpeed']
    
weather_2['AvgSpeed'] = pd.to_numeric(weather_2['AvgSpeed'], errors = 'coerce')

### Combine Data from Both Stations

We will check if there is any significant difference between value obtained from Station 1 and 2 based on their Pearson's correlation. If there are very high correlation, we will combine them by taking their average values. However, if there is significant difference, we will observe the particular columns in a greater details.

For 'Depart', 'Sunset', and 'Sunrise' columns, we will use Station 1 data as Station 2 does not provide any information. 

In [68]:
columns_from_station_1 = ['Depart', 'Sunset', 'Sunrise']
columns_to_compare = [column for column in \
                      weather_1.columns if column  not in columns_from_station_1 + ['Station']]


for column in columns_to_compare:
    corr = np.corrcoef(weather_1[column], weather_2[column])[1, 0]
    print("Pearson's correlation of column {} between the two stations is: {}".format(column, round(corr,3)))

Pearson's correlation of column Tmax between the two stations is: 0.987
Pearson's correlation of column Tmin between the two stations is: 0.978
Pearson's correlation of column Tavg between the two stations is: 0.99
Pearson's correlation of column DewPoint between the two stations is: 0.99
Pearson's correlation of column WetBulb between the two stations is: 0.994
Pearson's correlation of column Heat between the two stations is: 0.989
Pearson's correlation of column Cool between the two stations is: 0.983
Pearson's correlation of column PrecipTotal between the two stations is: 0.67
Pearson's correlation of column AvgSpeed between the two stations is: 0.951


Most correlation values showing > 0.95 correlation. Therefore, we will take the average of values from both stations and take it as single value. However, as Precipitation Total has differing value, we will retain the value for further analysis. 

In [69]:
# Create weather_combined dataframe to house the cleaned data and input average values from both Stations
weather_combined = pd.DataFrame()

columns_to_combine = [column for column in \
                      columns_to_compare if column  not in ['PrecipTotal']]

for column in columns_to_combine:
    weather_combined[column] = (weather_1[column] + weather_2[column])/2
weather_combined.head(2)

,Tmax,Tmin,Tavg,DewPoint,WetBulb,Heat,Cool,AvgSpeed
Date,,,,,,,,
2007-05-01,83.5,51.0,67.5,51.0,56.5,0.0,2.5,9.4
2007-05-02,59.5,42.5,51.5,42.0,47.0,13.5,0.0,13.4


In [70]:
# Include values from Station 1
for column in columns_from_station_1:
    weather_combined[column] = weather_1[column]
weather_combined.head(2)

,Tmax,Tmin,Tavg,DewPoint,WetBulb,Heat,Cool,AvgSpeed,Depart,Sunset,Sunrise
Date,,,,,,,,,,,
2007-05-01,83.5,51.0,67.5,51.0,56.5,0.0,2.5,9.4,14,1849,448
2007-05-02,59.5,42.5,51.5,42.0,47.0,13.5,0.0,13.4,-3,1850,447


In [71]:
# Include both precipitation record on both Stations

weather_combined['Precip_1'] = weather_1['PrecipTotal']
weather_combined['Precip_2'] = weather_2['PrecipTotal']
weather_combined.head(2)

,Tmax,Tmin,Tavg,DewPoint,WetBulb,Heat,Cool,AvgSpeed,Depart,Sunset,Sunrise,Precip_1,Precip_2
Date,,,,,,,,,,,,,
2007-05-01,83.5,51.0,67.5,51.0,56.5,0.0,2.5,9.4,14,1849,448,0.0,0.0
2007-05-02,59.5,42.5,51.5,42.0,47.0,13.5,0.0,13.4,-3,1850,447,0.0,0.0


In [72]:
week_of_year_generator(weather_combined)
weather_combined.head(2)

,Tmax,Tmin,Tavg,DewPoint,WetBulb,Heat,Cool,AvgSpeed,Depart,Sunset,Sunrise,Precip_1,Precip_2,week_of_year
Date,,,,,,,,,,,,,,
2007-05-01,83.5,51.0,67.5,51.0,56.5,0.0,2.5,9.4,14,1849,448,0.0,0.0,18
2007-05-02,59.5,42.5,51.5,42.0,47.0,13.5,0.0,13.4,-3,1850,447,0.0,0.0,18


## Spray Dataset

In [73]:
spray = read_csv_file(use_colab, 'spray.csv')
spray.head()

,Date,Time,Latitude,Longitude
0,2011-08-29,6:56:58 PM,42.391623,-88.089163
1,2011-08-29,6:57:08 PM,42.391348,-88.089163
2,2011-08-29,6:57:18 PM,42.391022,-88.089157
3,2011-08-29,6:57:28 PM,42.390637,-88.089158
4,2011-08-29,6:57:38 PM,42.390410,-88.088858


In [74]:
spray.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14835 entries, 0 to 14834
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       14835 non-null  object 
 1   Time       14251 non-null  object 
 2   Latitude   14835 non-null  float64
 3   Longitude  14835 non-null  float64
dtypes: float64(2), object(2)
memory usage: 463.7+ KB


In [75]:
datetime_setter(spray)
spray.index.unique()

DatetimeIndex(['2011-08-29', '2011-09-07', '2013-07-17', '2013-07-25',
               '2013-08-08', '2013-08-15', '2013-08-16', '2013-08-22',
               '2013-08-29', '2013-09-05'],
              dtype='datetime64[ns]', name='Date', freq=None)

**Observations:**
- No Data cleaning needs to be done on this dataset apart from setting index to datetime.
- Spray dataset only contains data from 2011 and 2013. This likely to be used only as a gauge of how effective spraying can be but not as the data to train our prediction model.

## File Export

In [76]:
train.to_csv('dataset/train_cleaned.csv')
test.to_csv('dataset/test_cleaned.csv')
weather_combined.to_csv('dataset/weather_cleaned.csv')
spray.to_csv('dataset/spray.csv')